In [ ]:
from imports import *
from PreProcessor import *

_PreProcessor = PreProcessor()


In [ ]:
function_names_train,function_segments_train = _PreProcessor.setupDeepLearning("./Dataset/teiid_dataset/train_CC",'CC','train')
descriptions_train, summaries_train = _PreProcessor.setupDeepLearning("./Dataset/teiid_dataset/train_UC",'UC','train')
function_names_test,function_segments_test = _PreProcessor.setupDeepLearning("./Dataset/teiid_dataset/test_CC",'CC','test')
descriptions_test, summaries_test = _PreProcessor.setupDeepLearning("./Dataset/teiid_dataset/test_UC",'UC','test')

function_names_train_errai,function_segments_train_errai = _PreProcessor.setupDeepLearning("./Dataset/errai_dataset/train_CC",'CC','train')
descriptions_train_errai, summaries_train_errai = _PreProcessor.setupDeepLearning("./Dataset/errai_dataset/train_UC",'UC','train')
function_names_test_errai,function_segments_test_errai = _PreProcessor.setupDeepLearning("./Dataset/errai_dataset/test_CC",'CC','test')
descriptions_test_errai, summaries_test_errai = _PreProcessor.setupDeepLearning("./Dataset/errai_dataset/test_UC",'UC','test')

function_names_train_jboss,function_segments_train_jboss = _PreProcessor.setupDeepLearning("./Dataset/jboss_dataset/train_CC",'CC','train')
descriptions_train_jboss, summaries_train_jboss = _PreProcessor.setupDeepLearning("./Dataset/jboss_dataset/train_UC",'UC','train')
function_names_test_jboss,function_segments_test_jboss = _PreProcessor.setupDeepLearning("./Dataset/jboss_dataset/test_CC",'CC','test')
descriptions_test_jboss, summaries_test_jboss = _PreProcessor.setupDeepLearning("./Dataset/jboss_dataset/test_UC",'UC','test')

In [ ]:
_PreProcessor.setUpUnknown(function_names_train,function_segments_train,"CC",'train')
_PreProcessor.setUpUnknown(descriptions_train,summaries_train,"UC",'train  ')

_PreProcessor.setUpUnknown(function_names_test,function_segments_test,"CC",'test')
_PreProcessor.setUpUnknown(descriptions_test,summaries_test,"UC",'test')

######################################################################################

_PreProcessor.setUpUnknown(function_names_train_errai,function_segments_train_errai,"CC",'train')
_PreProcessor.setUpUnknown(descriptions_train_errai,summaries_train_errai,"UC",'train')

_PreProcessor.setUpUnknown(function_names_test_errai,function_segments_test_errai,"CC",'test')
_PreProcessor.setUpUnknown(descriptions_test_errai,summaries_test_errai,"UC",'test')

#####################################################################################

_PreProcessor.setUpUnknown(function_names_train_jboss,function_segments_train_jboss,"CC",'train')
_PreProcessor.setUpUnknown(descriptions_train_jboss,summaries_train_jboss,"UC",'train')

_PreProcessor.setUpUnknown(function_names_test_jboss,function_segments_test_jboss,"CC",'test')
_PreProcessor.setUpUnknown(descriptions_test_jboss,summaries_test_jboss,"UC",'test')

In [ ]:
with open('./pickles/DeepLearning/function_names_train.pkl', 'wb') as f:
       pickle.dump(function_names_train, f)
with open('./pickles/DeepLearning/function_segments_train.pkl', 'wb') as f:
        pickle.dump(function_segments_train, f)
with open('./pickles/DeepLearning/descriptions_train.pkl', 'wb') as f:
        pickle.dump(descriptions_train, f)
with open('./pickles/DeepLearning/summaries_train.pkl', 'wb') as f:
        pickle.dump(summaries_train, f)

with open('./pickles/DeepLearning/Vocab.pkl', 'wb') as f:
        pickle.dump(_PreProcessor.Vocab, f)

with open('./pickles/DeepLearning/function_names_test.pkl', 'wb') as f:
       pickle.dump(function_names_test, f)
with open('./pickles/DeepLearning/function_segments_test.pkl', 'wb') as f:
        pickle.dump(function_segments_test, f)
with open('./pickles/DeepLearning/descriptions_test.pkl', 'wb') as f:
        pickle.dump(descriptions_test, f)
with open('./pickles/DeepLearning/summaries_test.pkl', 'wb') as f:
        pickle.dump(summaries_test, f)
        
with open('./pickles/DeepLearning/function_names_train_errai.pkl', 'wb') as f:
       pickle.dump(function_names_train_errai, f)
with open('./pickles/DeepLearning/function_segments_train_errai.pkl', 'wb') as f:
        pickle.dump(function_segments_train_errai, f)
with open('./pickles/DeepLearning/descriptions_train_errai.pkl', 'wb') as f:
        pickle.dump(descriptions_train_errai, f)
with open('./pickles/DeepLearning/summaries_train_errai.pkl', 'wb') as f:
        pickle.dump(summaries_train_errai, f)

with open('./pickles/DeepLearning/function_names_test_errai.pkl', 'wb') as f:
        pickle.dump(function_names_test_errai, f)
with open('./pickles/DeepLearning/function_segments_test_errai.pkl', 'wb') as f:
       pickle.dump(function_segments_test_errai, f)
with open('./pickles/DeepLearning/descriptions_test_errai.pkl', 'wb') as f:
        pickle.dump(descriptions_test_errai, f)
with open('./pickles/DeepLearning/summaries_test_errai.pkl', 'wb') as f:
        pickle.dump(summaries_test_errai, f)

with open('./pickles/DeepLearning/function_names_train_jboss.pkl', 'wb') as f:
       pickle.dump(function_names_train_jboss, f)
with open('./pickles/DeepLearning/function_segments_train_jboss.pkl', 'wb') as f:
        pickle.dump(function_segments_train_jboss, f)
with open('./pickles/DeepLearning/descriptions_train_jboss.pkl', 'wb') as f:
        pickle.dump(descriptions_train_jboss, f)
with open('./pickles/DeepLearning/summaries_train_jboss.pkl', 'wb') as f:
        pickle.dump(summaries_train_jboss, f)

with open('./pickles/DeepLearning/function_names_test_jboss.pkl', 'wb') as f:
        pickle.dump(function_names_test_jboss, f)
with open('./pickles/DeepLearning/function_segments_test_jboss.pkl', 'wb') as f:
       pickle.dump(function_segments_test_jboss, f)
with open('./pickles/DeepLearning/descriptions_test_jboss.pkl', 'wb') as f:
        pickle.dump(descriptions_test_jboss, f)
with open('./pickles/DeepLearning/summaries_test_jboss.pkl', 'wb') as f:
        pickle.dump(summaries_test_jboss, f)

In [ ]:
function_names_train = np.load('./pickles/DeepLearning/function_names_train.pkl',allow_pickle=True)
function_segments_train = np.load('./pickles/DeepLearning/function_segments_train.pkl',allow_pickle=True)
descriptions_train = np.load('./pickles/DeepLearning/descriptions_train.pkl',allow_pickle=True)
summaries_train = np.load('./pickles/DeepLearning/summaries_train.pkl',allow_pickle=True)

function_names_test = np.load('./pickles/DeepLearning/function_names_test.pkl',allow_pickle=True)
function_segments_test = np.load('./pickles/DeepLearning/function_segments_test.pkl',allow_pickle=True)
descriptions_test = np.load('./pickles/DeepLearning/descriptions_test.pkl',allow_pickle=True)
summaries_test = np.load('./pickles/DeepLearning/summaries_test.pkl',allow_pickle=True)

function_names_train_errai = np.load('./pickles/DeepLearning/function_names_train_errai.pkl',allow_pickle=True)
function_segments_train_errai = np.load('./pickles/DeepLearning/function_segments_train_errai.pkl',allow_pickle=True)
descriptions_train_errai = np.load('./pickles/DeepLearning/descriptions_train_errai.pkl',allow_pickle=True)
summaries_train_errai = np.load('./pickles/DeepLearning/summaries_train_errai.pkl',allow_pickle=True)

function_names_test_errai = np.load('./pickles/DeepLearning/function_names_test_errai.pkl',allow_pickle=True)
function_segments_test_errai = np.load('./pickles/DeepLearning/function_segments_test_errai.pkl',allow_pickle=True)
descriptions_test_errai = np.load('./pickles/DeepLearning/descriptions_test_errai.pkl',allow_pickle=True)
summaries_test_errai = np.load('./pickles/DeepLearning/summaries_test_errai.pkl',allow_pickle=True)

function_names_train_jboss = np.load('./pickles/DeepLearning/function_names_train_jboss.pkl',allow_pickle=True)
function_segments_train_jboss = np.load('./pickles/DeepLearning/function_segments_train_jboss.pkl',allow_pickle=True)
descriptions_train_jboss = np.load('./pickles/DeepLearning/descriptions_train_jboss.pkl',allow_pickle=True)
summaries_train_jboss = np.load('./pickles/DeepLearning/summaries_train_jboss.pkl',allow_pickle=True)

function_names_test_jboss = np.load('./pickles/DeepLearning/function_names_test_jboss.pkl',allow_pickle=True)
function_segments_test_jboss = np.load('./pickles/DeepLearning/function_segments_test_jboss.pkl',allow_pickle=True)
descriptions_test_jboss = np.load('./pickles/DeepLearning/descriptions_test_jboss.pkl',allow_pickle=True)
summaries_test_jboss = np.load('./pickles/DeepLearning/summaries_test_jboss.pkl',allow_pickle=True)

In [ ]:
# function_names_train.extend(function_names_train_errai); function_names_train.extend(function_names_train_jboss) 
# function_segments_train.extend(function_segments_train_errai); function_segments_train.extend(function_segments_train_jboss)

# descriptions_train.extend(descriptions_train_errai); descriptions_train.extend(descriptions_train_jboss)
# summaries_train.extend(summaries_train_errai); summaries_train.extend(summaries_train_jboss)

# function_names_test.extend(function_names_test_errai); function_names_test.extend(function_names_test_jboss)
# function_segments_test.extend(function_segments_test_errai); function_segments_test.extend(function_segments_test_jboss)

# descriptions_test.extend(descriptions_test_errai); descriptions_test.extend(descriptions_test_jboss)
# summaries_test.extend(summaries_test_errai); summaries_test.extend(summaries_test_jboss)

In [ ]:
CC_word2vec = _PreProcessor.word2VecProcessor(function_names_train, function_segments_train, 'CC')
UC_word2vec = _PreProcessor.word2VecProcessor(summaries_train, descriptions_train, 'UC')

word2vec_model = Word2Vec(sentences = CC_word2vec + UC_word2vec, vector_size=150, window=5, min_count=1, workers=4, epochs=10)

word2vec_model.save("./Dataset/teiid_dataset/UC_CC_WORD2VEC")
word2vec_model = Word2Vec.load("./Dataset/teiid_dataset/UC_CC_WORD2VEC")

word2vec_vocab = word2vec_model.wv.index_to_key

# Initialize an empty embedding matrix
embedding_matrix = np.zeros((len(word2vec_vocab) + 1, word2vec_model.vector_size))

# Fill the embedding matrix with the embeddings of each word
for i, word in enumerate(word2vec_vocab):
    embedding_vector = word2vec_model.wv[word]
    if embedding_vector is not None:
        embedding_matrix[i + 1] = embedding_vector

In [ ]:
with open('./pickles/DeepLearning/word2vec_vocab.pkl', 'wb') as f:
    pickle.dump(word2vec_vocab, f)
with open('./pickles/DeepLearning/embedding_matrix.pkl', 'wb') as f:
    pickle.dump(embedding_matrix, f)

In [ ]:
embedding_matrix = np.load('./Dataset/teiid_dataset/embedding_matrix.pkl',allow_pickle=True)
word2vec_vocab = np.load('./Dataset/teiid_dataset/word2vec_vocab.pkl',allow_pickle=True)

In [ ]:
print(word2vec_vocab)

### Converting dataset to indices

In [ ]:
_PreProcessor.vocabToIndex(word2vec_vocab)

#############################################################################

_PreProcessor.dataSetToIndex(function_names_train,function_segments_train,"CC")
_PreProcessor.dataSetToIndex(descriptions_train,summaries_train,"UC")

_PreProcessor.dataSetToIndex(function_names_test,function_segments_test,"CC")
_PreProcessor.dataSetToIndex(descriptions_test,summaries_test,"UC")

############################################################################

_PreProcessor.dataSetToIndex(function_names_train_errai,function_segments_train_errai,"CC")
_PreProcessor.dataSetToIndex(descriptions_train_errai,summaries_train_errai,"UC")

_PreProcessor.dataSetToIndex(function_names_test_errai,function_segments_test_errai,"CC")
_PreProcessor.dataSetToIndex(descriptions_test_errai,summaries_test_errai,"UC")

############################################################################

_PreProcessor.dataSetToIndex(function_names_train_jboss,function_segments_train_jboss,"CC")
_PreProcessor.dataSetToIndex(descriptions_train_jboss,summaries_train_jboss,"UC")

_PreProcessor.dataSetToIndex(function_names_test_jboss,function_segments_test_jboss,"CC")
_PreProcessor.dataSetToIndex(descriptions_test_jboss,summaries_test_jboss,"UC")

In [ ]:
x_train, labels = _PreProcessor.setUpLabels(function_names_train,function_segments_train,descriptions_train,summaries_train,'Dataset/teiid_dataset/train_modified.csv')
x_train_errai, labels_errai = _PreProcessor.setUpLabels(function_names_train_errai,function_segments_train_errai,descriptions_train_errai,summaries_train_errai,'Dataset/errai_dataset/train_modified.csv')
x_train_jboss, labels_jboss = _PreProcessor.setUpLabels(function_names_train_jboss,function_segments_train_jboss,descriptions_train_jboss,summaries_train_jboss,'Dataset/jboss_dataset/train_modified.csv')

In [ ]:
x_test, labels_test = _PreProcessor.setUpLabels(function_names_test,function_segments_test,descriptions_test,summaries_test,'Dataset/teiid_dataset/test_modified.csv')
x_test_errai, labels_test_errai = _PreProcessor.setUpLabels(function_names_test_errai,function_segments_test_errai,descriptions_test_errai,summaries_test_errai,'Dataset/errai_dataset/test_modified.csv')
x_test_jboss, labels_test_jboss = _PreProcessor.setUpLabels(function_names_test_jboss,function_segments_test_jboss,descriptions_test_jboss,summaries_test_jboss,'Dataset/jboss_dataset/test_modified.csv')

In [ ]:
x_train.extend(x_train_errai)
x_train.extend(x_train_jboss)

x_test.extend(x_test_errai)
x_test.extend(x_test_jboss)

labels.extend(labels_errai)
labels.extend(labels_jboss)

labels_test.extend(labels_test_errai)
labels_test.extend(labels_test_jboss)

In [ ]:
with open('./pickles/DeepLearning/x_train.pkl', 'wb') as f:
       pickle.dump(x_train, f)
with open('./pickles/DeepLearning/labels.pkl', 'wb') as f:
        pickle.dump(labels, f)

In [ ]:
with open('./pickles/DeepLearning/x_test.pkl', 'wb') as f:
       pickle.dump(x_test, f)
with open('./pickles/DeepLearning/labels_test.pkl', 'wb') as f:
       pickle.dump(labels_test, f)

In [ ]:
x_train = np.load('./pickles/DeepLearning/x_train.pkl',allow_pickle=True)
labels = np.load('./pickles/DeepLearning/labels.pkl',allow_pickle=True)

In [ ]:
x_test = np.load('./pickles/DeepLearning/x_test.pkl',allow_pickle=True)
labels_test = np.load('./pickles/DeepLearning/labels_test.pkl',allow_pickle=True)

In [227]:
x_train_small = []
label_small = []
counter_1 = 0
counter_0 = 0
while(counter_1 < 11 or counter_0 < 11):
    index = random.randint(0, len(x_train))

    if labels[index] == 1 and counter_1 < 11 :
        x_train_small.append(x_train[index])
        label_small.append(1)
        counter_1 +=1

    elif labels[index] == 0 and counter_0 < 11 :
        x_train_small.append(x_train[index])
        label_small.append(0)
        counter_0 +=1
    

In [ ]:
def calculate_pos_weights(class_counts):
    pos_weights = np.ones_like(class_counts)
    neg_counts = [len(labels)-pos_count for pos_count in class_counts]
    for cdx, (pos_count, neg_count) in enumerate(zip(class_counts,  neg_counts)):
        pos_weights[cdx] = neg_count / (pos_count + 1e-5)

    return torch.as_tensor(pos_weights, dtype=torch.float)

class_counts = [len(labels)-sum(labels), sum(labels)]
pos_weights = calculate_pos_weights(class_counts)

In [ ]:
print(x_train_small)
print(label_small)

In [228]:
class TracibilityLinkDataset(Dataset):

    def __init__(self, x_train, y_train):
        

        function_names__, function_segments__, descriptions__ , summaries__ = zip(*x_train)
        function_names_padded = pad_sequence(function_names__, batch_first=True, padding_value=0)
        function_segments_padded = pad_sequence(function_segments__, batch_first=True, padding_value = 0)
        descriptions_padded = pad_sequence(descriptions__, batch_first=True, padding_value=0)
        summaries_padded = pad_sequence(summaries__, batch_first=True, padding_value = 0)

        x_train_padded = list()
        for data in zip(function_names_padded, function_segments_padded, descriptions_padded, summaries_padded):
            x_train_padded.append(list(data))


        self.tensor_x_train = x_train_padded
        self.tensor_y_train = torch.tensor(y_train)



    # len and get item are very important --> used by dataloader
    def __len__(self):
        return len(self.tensor_y_train)

    def __getitem__(self, idx):
        return self.tensor_x_train[idx], self.tensor_y_train[idx]


In [229]:
dataset_train_small = TracibilityLinkDataset(x_train_small, label_small)

In [ ]:
dataset_test = TracibilityLinkDataset(x_test, labels_test)

In [255]:
class DLModel(nn.Module):

  def __init__(self,embedding_matrix : np.array, embedding_dim : tuple,name_size: int, seg_size:int, summary_size:int,descript_size:int, hidden_size:int = 64, classes: int = 2):
      super(DLModel, self).__init__()

      # first embedding layer: input size --> vocab size, output size --> feature vector of vocab size(one hot encoding), weigths --> word2vec
      # matrix (vocab * vocab)
      # embedding_dim[0] -> vocab size, embedding_dim[1] -> 150 defined in word2vec
      self.embedding = nn.Embedding(num_embeddings = embedding_dim[0], embedding_dim = embedding_dim[1], _weight = torch.tensor(embedding_matrix))
      
      self.lstm_CC = nn.LSTM(embedding_dim[1],hidden_size,batch_first=True, bidirectional=False)
      self.lstm_post_flatten = nn.Linear((name_size+seg_size)*hidden_size+(summary_size+descript_size)*hidden_size,hidden_size)

      self.linear = nn.Linear(hidden_size,classes)

      self.softmax = nn.Softmax(dim=1)
      # function_names_lstm.shape =  torch.Size([2, 4, 128])
      # function_segments_lstm.shape =  torch.Size([2, 165, 128])
      # descriptions_lstm.shape =  torch.Size([2, 200, 128])
      # summaries_lstm.shape =  torch.Size([2, 12, 128])
      # summaries_descrip_flatten.shape =  torch.Size([2, 27136])
      # name_seg_flatten.shape =  torch.Size([2, 21632])
      # torch.Size([2])
      # torch.Size([2])
      
      # self.cosine_similarity = nn.CosineSimilarity(dim=1) # code = 1*(vocab_size)*(feature_vector) , req = code = 1*1000*200 

      # self.sigmoid = nn.Sigmoid()
    
  def forward(self, function_names, function_segments, descriptions, summaries ):
    #function_names.shape --> batch_no * no_tokens* 150
    
    function_names_embeddings  = self.embedding(function_names)
    function_segments_embeddings = self.embedding(function_segments)
    descriptions_embedding  = self.embedding(descriptions)
    summaries_embedding = self.embedding(summaries)
    
    #function_names.shape --> batch_no * no_tokens* 150
    
    
    function_names_lstm,_ = self.lstm_CC (function_names_embeddings.float())
    function_segments_lstm,_ = self.lstm_CC (function_segments_embeddings.float())
    descriptions_lstm,_ = self.lstm_CC (descriptions_embedding.float())
    summaries_lstm,_ = self.lstm_CC (summaries_embedding.float())
    
    # print("function_names_lstm.shape = ",function_names_lstm.shape)
    # print("function_segments_lstm.shape = ",function_segments_lstm.shape)
    # print("descriptions_lstm.shape = ",descriptions_lstm.shape)
    # print("summaries_lstm.shape = ",summaries_lstm.shape)

    #function_names.shape --> batch_no * no_tokens* 150

    concatenated_name_seg = torch.cat((function_names_lstm, function_segments_lstm), dim = 1)
    summaries_descrip_seg = torch.cat((descriptions_lstm, summaries_lstm), dim = 1)


    summaries_descrip_flatten = summaries_descrip_seg.view(summaries_descrip_seg.shape[0],-1)
    name_seg_flatten = concatenated_name_seg.view(concatenated_name_seg.shape[0],-1) # batch_no * (feature_vector * no_tokens)


    lstm_post_output = self.lstm_post_flatten(torch.cat([name_seg_flatten,summaries_descrip_flatten],axis=1).float())
    # print("summaries_descrip_flatten.shape = ",summaries_descrip_flatten.shape)
    # print("name_seg_flatten.shape = ",name_seg_flatten.shape)

    # num_repeats = (name_seg_flatten.shape[1] + summaries_descrip_flatten.shape[1] - 1) // summaries_descrip_flatten.shape[1] 

    # inreasing th size of the req document to be the same as the code documet to allow for cosine simiraity 
    # summaries_descrip_flatten_tiled = summaries_descrip_flatten.repeat((1,num_repeats))
    # summaries_descrip_flatten_tiled_segmented = summaries_descrip_flatten_tiled[:,:name_seg_flatten.shape[1]]
    
    linear_output = self.linear(lstm_post_output)
    
    # softmax_output = self.softmax(linear_output)

    # consine_sim_code_req = self.cosine_similarity(name_seg_flatten,summaries_descrip_flatten_tiled_segmented)

    return linear_output
#     concatenated_tensor = torch.cat([function_names_lstm_output[0].reshape(batch_nu,-1), function_segments_lstm_output[0].reshape(batch_nu,-1), descriptions_lstm_output[0].reshape(batch_nu,-1) , summaries_lstm_output[0].reshape(batch_nu,-1)],axis=1)
#     linear_output = self.linear(concatenated_tensor)
    
#     print("concatenated_tensor= ",linear_output.shape)

#     function_names_attention = self.attention_layer_CC(function_names_embeddings.float(),function_names_embeddings.float(),function_names_embeddings.float())

#     function_segments_attention = self.attention_layer_CC(function_segments_embeddings.float(),function_segments_embeddings.float(),function_segments_embeddings.float())
   
#     descriptions_attention = self.attention_layer_UC(descriptions_embedding.float(),descriptions_embedding.float(),descriptions_embedding.float())

#     summaries_attention = self.attention_layer_UC(summaries_embedding.float(),summaries_embedding.float(),summaries_embedding.float())

#     concatenated_name_seg = torch.cat((function_names_attention[0], function_segments_attention[0]), dim = 1)

#     summaries_descrip_seg = torch.cat((descriptions_attention[0], summaries_attention[0]), dim = 1)

#     names_seg_conv = self.conv_layer_2d(concatenated_name_seg.unsqueeze(1))

#     # summaries_descrip_conv = self.conv_layer_2d(summaries_descrip_seg.unsqueeze(1))
#     # print("summaries_descrip_seg.shape = ",summaries_descrip_seg.shape)

# #     name_seg = self.pool_layer_2d (names_seg_conv)
#     # print("name_seg.shape = " , name_seg.shape)

#     summaries_descrip_flatten = summaries_descrip_seg.view(summaries_descrip_seg.shape[0],-1)
#     name_seg_flatten = names_seg_conv.squeeze(1).view(names_seg_conv.shape[0],-1) 


#     num_repeats = (name_seg_flatten.shape[1] + summaries_descrip_flatten.shape[1] - 1) // summaries_descrip_flatten.shape[1] 

#     #inreasing th size of the req document to be the same as the code documet to allow for cosine simiraity 
#     summaries_descrip_flatten_tiled = summaries_descrip_flatten.repeat((1,num_repeats))
#     summaries_descrip_flatten_tiled_segmented = summaries_descrip_flatten_tiled[:,:name_seg_flatten.shape[1]]

#     consine_sim_code_req = self.cosine_similarity(name_seg_flatten,summaries_descrip_flatten_tiled_segmented)
    
#     consine_sim_code_req_0_1 = self.sigmoid (consine_sim_code_req)
#     summary_desc = self.pool_layer_2d (summaries_descrip_seg)
#     print("summary_desc.shape = ",summary_desc.shape)

#     print(torch.cuda.memory_reserved()/1024**3)
    
#     print("consine_sim_code_req.shape", consine_sim_code_req.shape)
#     print(consine_sim_code_req.shape)
    

In [256]:
class JaccardLoss(torch.nn.Module):
    def __init__(self):
        super(JaccardLoss, self).__init__()

    def forward(self, input, target):
        intersection = torch.sum(input * target)
        union = torch.sum(input) + torch.sum(target) - intersection
        jaccard = (intersection + 1e-6) / (union + 1e-6)  # Adding epsilon for numerical stability
        return 1 - jaccard  # Minimizing 1 - Jaccard is equivalent to maximizing Jaccard

In [257]:
from torch.nn.utils.rnn import pad_sequence

def weighted_binary_cross_entropy(output, target):
        
#     loss = weights[1] *(target * torch.log(output)) + \
#            weights[0] * ((1 - target) * torch.log(1 - output))
#     output --> kber ,loss 
    loss = (target * torch.log(output)) + \
            ((1 - target) * torch.log(1 - output))

    return torch.mean(loss)

def customCollate(batch: list):
    # batch --> tuple (x,y)
    # x --> list of 4 lists
    x_batch, y_batch = zip(*batch)

    function_names, function_segments, descriptions , summaries = zip(*x_batch)

    function_names_padded = pad_sequence(function_names, batch_first=True, padding_value=0)
    function_segments_padded = pad_sequence(function_segments, batch_first=True, padding_value = 0)
    descriptions_padded = pad_sequence(descriptions, batch_first=True, padding_value=0)
    summaries_padded = pad_sequence(summaries, batch_first=True, padding_value = 0)

    return  function_names_padded, function_segments_padded, descriptions_padded, summaries_padded, y_batch

def train(model, train_dataset, batch_size=5, epochs=30 ,learning_rate=0.0001):
    
    weight = torch.as_tensor([1,50], dtype=torch.float)
    # (1) create the dataloader of the training set IMPORTANT (make the shuffle=True)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
    
    
    # (2) make the criterion cross entropy loss
    criterion = torch.nn.CrossEntropyLoss()
    # (3) create the optimizer (Adam)
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu") 
#     device="cpu"
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()
   

    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0
        n_samples = 0
        y_size = 0
        true_pos = 0
        false_neg = 0
        false_pos = 0 
        true_neg = 0
        for x_train, y_batch in tqdm(train_dataloader):
            # torch.cuda.empty_cache()     
            function_names_padded_tensor = torch.tensor(x_train[0], device=device)
            function_segments_padded_tensor = torch.tensor(x_train[1], device=device)
            descriptions_padded_tensor = torch.tensor(x_train[2], device=device)
            fsummaries_padded_tensor = torch.tensor(x_train[3], device=device)
            y_batch_tensor = torch.tensor(y_batch,dtype = torch.float32,device=device)
            
            output = model.forward(function_names_padded_tensor, function_segments_padded_tensor, descriptions_padded_tensor, fsummaries_padded_tensor)
#             _,predicted=torch.max(output)  #512 * 104  for every word in every sentence we choose one tag form the seventen tag 
            output_0_1 = torch.tensor((output > 0.5), dtype=torch.float32)
            
            # for i,value in enumerate(output_0_1): #calc the fals negtive
            #     if value == 0 and  y_batch_tensor[i] == 1:
            #       false_neg+=1
            #     elif value == 1 and  y_batch_tensor[i] == 1:
            #       true_pos+=1
            #     elif value == 1 and  y_batch_tensor[i] == 0:
            #       false_pos += 1
            #     else:
            #       true_neg += 1
            
#             weights = torch.where(y_batch_tensor == 0, torch.tensor(class_weights[0]).to(device), torch.tensor(class_weights[1]).to(device)).to(device)
            
            _,predicted=torch.max(output,dim=1)
            batch_loss = criterion(output,y_batch_tensor.long())
            # batch_loss = criterion(output, y_batch_tensor)
            # torch.cuda.empty_cache()
            total_loss_train += batch_loss
            # print(torch.cuda.memory_reserved()/1024**3)
              

            # (7) loss calculation (you need to think in this part how to calculate the loss correctly)
            
    #       (9) calculate the batch accuracy (just add the number of correct predictions)
            acc = (predicted==y_batch_tensor).sum().item()
            n_samples += y_batch_tensor.size(0)
            total_acc_train += acc


            optimizer.zero_grad()

    #       (11) do the backward pass
            batch_loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    #       (12) update the weights with your optimizer
            optimizer.step()
            # (10) zero your gradients
            
    #   epoch loss
        epoch_loss = total_loss_train / n_samples

    #   (13) calculate the accuracy
        epoch_acc =100*total_acc_train/n_samples

        print(
           f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
           | Train Accuracy: {epoch_acc}\n')
        print("true_pos = ", true_pos)
        print("false_neg = ",false_neg)
        print("false_pos = ",false_pos)
        print("true_neg = ", true_neg)
        

  ##############################################################################################################

In [258]:
model = DLModel(embedding_matrix, embedding_matrix.shape,len(dataset_train_small[0][0][0]),len(dataset_train_small[0][0][1]),len(dataset_train_small[0][0][2]),len(dataset_train_small[0][0][3]), hidden_size=64, classes=2)
train(model, dataset_train_small)


  0%|          | 0/5 [00:00<?, ?it/s]C:\Users\HP\AppData\Local\Temp\ipykernel_14616\3258290520.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  function_names_padded_tensor = torch.tensor(x_train[0], device=device)
C:\Users\HP\AppData\Local\Temp\ipykernel_14616\3258290520.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  function_segments_padded_tensor = torch.tensor(x_train[1], device=device)
C:\Users\HP\AppData\Local\Temp\ipykernel_14616\3258290520.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  descriptions_padded_tensor = 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1660x150 and 600x128)

In [ ]:
# # import matplotlib.pyplot as plt
# model = DLModel(embedding_matrix, embedding_matrix.shape)
# weight = torch.as_tensor([50], dtype=torch.float)
# def customCollate(batch: list):
#     # batch --> tuple (x,y)
#     # x --> list of 4 lists
#     x_batch, y_batch = zip(*batch)
#     function_names, function_segments, descriptions , summaries = zip(*x_batch)

#     function_names_padded = pad_sequence(function_names, batch_first=True, padding_value=0)
#     function_segments_padded = pad_sequence(function_segments, batch_first=True, padding_value = 0)
#     descriptions_padded = pad_sequence(descriptions, batch_first=True, padding_value=0)
#     summaries_padded = pad_sequence(summaries, batch_first=True, padding_value = 0)

#     return  function_names_padded, function_segments_padded, descriptions_padded, summaries_padded, y_batch

# criterion = torch.nn.BCEWithLogitsLoss(pos_weight=weight)
# optimizer = torch.optim.Adam(model.parameters())
# # use_cuda = torch.cuda.is_available()
# #device = torch.device("cuda" if use_cuda else "cpu") '
# device="cpu"
# # if use_cuda:
# #     model = model.cuda()
# #     criterion = criterion.cuda()
    
# train_dataloader = torch.utils.data.DataLoader(dataset_train, 5, shuffle=True, collate_fn=customCollate)

# # train(model, dataset_train)
# # model.load_state_dict(torch.load('ModelBCEWithLogitsLoss_attentionLayer_1_epoch_weight60.pth'))
# # model.train()
# from ignite.engine import Engine, Events
# from ignite.handlers import FastaiLRFinder

# lr_finder = FastaiLRFinder()

# trainer=Engine(train_step)
# to_save = {"model": model, "optimizer": optimizer}

# with lr_finder.attach(trainer, to_save=to_save) as trainer_with_lr_finder:
#     trainer_with_lr_finder.run(train_dataloader)
    


# # def custom_plot(lr_finder, save_path='lr_finder_plot.png', figsize=(10, 6)):
# #     fig, ax = plt.subplots(figsize=figsize)
# #     lrs = lr_finder.get_results()["lr"]
# #     losses = lr_finder.get_results()["loss"]
# #     ax.plot(lrs, losses)
# #     ax.set_xscale("log")
# #     ax.set_xlabel("Learning rate")
# #     ax.set_ylabel("Loss")
# #     ax.set_title("Learning rate vs. Loss")
# #     plt.savefig(save_path)
# #     plt.show()

# # # Use the custom plot function
# # lr_finder.get_results()
# # custom_plot(lr_finder, save_path='lr_finder_plot.png', figsize=(12, 8))
# # suggested_lr = lr_finder.lr_suggestion()
# # print(f"Suggested learning rate: {suggested_lr}")

In [ ]:
'''
ModelBCEWithLogitsLoss_attentionLayer_1_epoch_weight60 --> 
Test:
true_pos =  41
false_neg =  25
false_pos =  821
true_neg =  940

Test Accuracy: 53.694580078125

Test Recall: 0.6212121212121212

Test precesion: 0.04756380510440835
--------------------------------------------
ModelBCEWithLogitsLoss_attentionLayer_1_epoch_weight45 --> 
Test:
true_pos =  6
false_neg =  60
false_pos =  185
true_neg =  1576

Test Accuracy: 86.59004211425781

Test Recall: 0.09090909090909091

Test precesion: 0.031413612565445025
------------------------------------------------
ModelBCEWithLogitsLoss_attentionLayer_1_epoch_weight50
true_pos =  32
false_neg =  34
false_pos =  772
true_neg =  989

Test Accuracy: 55.88396453857422

Test Recall: 0.48484848484848486

Test precesion: 0.03980099502487562
------------------------------------------------
ModelBCEWithLogitsLoss_attentionLayer_1_epoch_weight70 -->

true_pos =  37
false_neg =  29
false_pos =  971
true_neg =  790

Test Accuracy: 45.26546096801758

Test Recall: 0.5606060606060606

Test precesion: 0.03670634920634921
-----------------------------------------------------
ModelBCEWithLogitsLoss_attentionLayer_1_epoch_weight60 -->

true_pos =  41
false_neg =  25
false_pos =  821
true_neg =  940

Test Accuracy: 53.694580078125

Test Recall: 0.6212121212121212

Test precesion: 0.04756380510440835
----------------------------------------------------------
NewModelBCEWithLogitsLoss_attentionLayer_1_epoch_weight100

true_pos =  40
false_neg =  26
false_pos =  993
true_neg =  768

Test Accuracy: 44.22550582885742

Test Recall: 0.6060606060606061

Test precesion: 0.03872216844143272
--------------------------------------------------------
ModelBCEWithLogitsLoss_attentionLayer_20_epoch_weight50

true_pos =  66
false_neg =  0
false_pos =  1761
true_neg =  0

Test Accuracy: 3.6124794483184814

Test Recall: 1.0

Test precesion: 0.0361247947454844
'''

In [ ]:
torch.save(model.state_dict(), 'ModelBCEWithLogitsLoss_preprocessingUpdated_lstm_1_epoch_weight50.pth')

In [ ]:
model.load_state_dict(torch.load('/kaggle/input/functions-divided.pth'))
model.eval()

In [ ]:
def evaluate(model, test_dataset,batch_size=5):
  """
  This function takes a NER model and evaluates its performance (accuracy) on a test data
  Inputs:
  - model: a NER model
  - test_dataset: dataset of type NERDataset
  """
  ########################### TODO: Replace the Nones in the following code ##########################

  # (1) create the test data loader
  test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, collate_fn=customCollate , shuffle=False)

  total_acc_test = 0
  n_samples = 0

  use_cuda = torch.cuda.is_available()
    
  device = torch.device("cuda" if use_cuda else "cpu") 
    # device="cpu"
  if use_cuda:
        model = model.cuda()
  with torch.no_grad():
    false_neg = 0
    true_pos = 0
    false_pos = 0
    true_neg = 0
    for function_names_padded, function_segments_padded, descriptions_padded, summaries_padded, y_batch in tqdm(test_dataloader):
            function_names_padded_tensor = torch.tensor(function_names_padded, device=device)
            function_segments_padded_tensor = torch.tensor(function_segments_padded, device=device)
            descriptions_padded_tensor = torch.tensor(descriptions_padded, device=device)
            fsummaries_padded_tensor = torch.tensor(summaries_padded, device=device)
            y_batch_tensor = torch.tensor(y_batch,dtype = torch.float32,device=device)
#             y_batch_tensor = torch.where(y_batch_tensor == 0 , torch.tensor(-1).to(device), torch.tensor(1).to(device)).to(device)
            output = model.forward(function_names_padded_tensor, function_segments_padded_tensor, descriptions_padded_tensor, fsummaries_padded_tensor)
            output_0_1 = torch.where(output < 0, torch.tensor(0).to(device), torch.tensor(1).to(device)).to(device)
#             print(output)
            for i,value in enumerate(output_0_1): #calc the fals negtive
                if value == 0 and  y_batch_tensor[i] == 1:
                  false_neg+=1
                elif value == 1 and  y_batch_tensor[i] == 1:
                  true_pos+=1
                elif value == 1 and  y_batch_tensor[i] == 0:
                  false_pos += 1
                else:
                  true_neg += 1
                
            n_true = sum((output_0_1 == y_batch_tensor))
            n_samples += y_batch_tensor.size(0)
            total_acc_test += n_true
       
    # (6) calculate the over all accuracy
    acc_test =100*total_acc_test/n_samples
  ##################################################################################################
    print("true_pos = ", true_pos)
    print("false_neg = ",false_neg)
    print("false_pos = ",false_pos)
    print("true_neg = ", true_neg)

    recall = true_pos / (true_pos+false_neg)
    precesion =  true_pos / (true_pos+false_pos)
#     f1Score = (2*precesion*recall)/(precesion+recall)
    print(f'\nTest Accuracy: {acc_test}')
    print(f'\nTest Recall: {recall}') 
    print(f'\nTest precesion: {precesion}') 
#     print(f'\nF1 Score: {f1Score}')


In [ ]:
evaluate(model, dataset_test)